<a href="https://colab.research.google.com/github/samko5sam/programming-language-class/blob/main/0410_TextProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install jieba
!pip install snownlp

In [2]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [3]:
from google.colab import auth
import gspread
from google.auth import default

In [24]:
import os
df = pd.DataFrame()

# 讀取遊戲題目建議資料

if os.path.exists('wis.csv'):
    df = pd.read_csv('wis.csv')
    if not df.empty:
        print("DataFrame has data.")
    else:
        print("DataFrame is empty.")
else:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1oJvpkS8p3nU36RM-ujACzz1xL8z0K7z0T5q-QNTovzQ/edit?usp=sharing').get_worksheet(0)
    dicts = dicts.get_all_records()
    df = pd.DataFrame(dicts)

df

DataFrame has data.


,Submission ID,Respondent ID,Submitted at,type,word1,word2,platform,lang,model,分類,字詞1,字詞2,題目說明,題目改進說明
0,7Aqepa,94yBXK,2024-12-30 11:36:46,改進現有題目,臭豆腐,狗屎,android,zh,SM-X110,改進現有題目,臭豆腐,狗屎,NaN,狗屎寫作咗個｛屎｝字（糞）
1,JPOAVX,XQdgvP,2024-12-31 19:41:30,新增題目,低音長號,低音號,android,zh,Pixel 9,新增題目,低音長號,低音號,NaN,NaN
2,oPGB1O,y1boo4,2025-01-01 09:06:22,改進現有題目,草莓,鳳梨,android,zh,SM-A5360,改進現有題目,草莓,番茄,NaN,跟鳳梨毫無關係
3,r4PG6v,VNad0E,2025-01-01 11:07:33,改進現有題目,喉糖,口香糖,android,zh,SM-X210,改進現有題目,櫻花,薰衣草,NaN,我第一次看到自己竟然可以出題目欸
4,vZ4qG4,A9gPBz,2025-01-03 15:05:50,新增題目,你／妳是平民,你／妳是臥底,ios,zh,"iPhone13,4",新增題目,你／妳是平民,你／妳是臥底,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0pRg00,95jRVE,2025-03-09 05:34:40,新增題目,麵包,蛋糕,android,zh,XQ-AT52,新增題目,麵包,蛋糕,NaN,NaN
60,M9pl78,95jRVE,2025-03-09 05:42:05,新增題目,西瓜,哈密瓜,android,zh,XQ-AT52,新增題目,西瓜,哈密瓜,NaN,NaN
61,LQLBR2,95jRVE,2025-03-09 05:47:34,新增題目,肉包,水餃,android,zh,XQ-AT52,新增題目,肉包,水餃,NaN,NaN
62,q2lZYd,Dy69Jj,2025-03-09 08:30:52,新增題目,阿諾絲 波魯迪戈烏多,阿諾斯 波魯迪戈烏多,ios,zh,"iPhone11,6",新增題目,阿諾絲 波魯迪戈烏多,阿諾斯 波魯迪戈烏多,大便又來了\n,NaN


In [25]:
import numpy as np
# 只保留中文字
def extract_chinese(text):
    if type(text) == float:
        return ''
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))
df['題目改進說明'] = df['題目改進說明'].apply(extract_chinese)
# 使用 Jieba 斷詞
df['Tokenized'] = df['題目改進說明'].apply(lambda x: list(jieba.cut(x, HMM=True)))
# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]
df

,Submission ID,Respondent ID,Submitted at,type,word1,word2,platform,lang,model,分類,字詞1,字詞2,題目說明,題目改進說明,Tokenized
0,7Aqepa,94yBXK,2024-12-30 11:36:46,改進現有題目,臭豆腐,狗屎,android,zh,SM-X110,改進現有題目,臭豆腐,狗屎,NaN,狗屎寫作咗個屎字糞,"[狗屎, 寫作, 咗, 個, 屎, 字, 糞]"
1,JPOAVX,XQdgvP,2024-12-31 19:41:30,新增題目,低音長號,低音號,android,zh,Pixel 9,新增題目,低音長號,低音號,NaN,,[]
2,oPGB1O,y1boo4,2025-01-01 09:06:22,改進現有題目,草莓,鳳梨,android,zh,SM-A5360,改進現有題目,草莓,番茄,NaN,跟鳳梨毫無關係,"[跟, 鳳梨毫, 無關, 係]"
3,r4PG6v,VNad0E,2025-01-01 11:07:33,改進現有題目,喉糖,口香糖,android,zh,SM-X210,改進現有題目,櫻花,薰衣草,NaN,我第一次看到自己竟然可以出題目欸,"[我, 第一次, 看到, 自己, 竟然, 可以, 出題, 目, 欸]"
4,vZ4qG4,A9gPBz,2025-01-03 15:05:50,新增題目,你／妳是平民,你／妳是臥底,ios,zh,"iPhone13,4",新增題目,你／妳是平民,你／妳是臥底,NaN,,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0pRg00,95jRVE,2025-03-09 05:34:40,新增題目,麵包,蛋糕,android,zh,XQ-AT52,新增題目,麵包,蛋糕,NaN,,[]
60,M9pl78,95jRVE,2025-03-09 05:42:05,新增題目,西瓜,哈密瓜,android,zh,XQ-AT52,新增題目,西瓜,哈密瓜,NaN,,[]
61,LQLBR2,95jRVE,2025-03-09 05:47:34,新增題目,肉包,水餃,android,zh,XQ-AT52,新增題目,肉包,水餃,NaN,,[]
62,q2lZYd,Dy69Jj,2025-03-09 08:30:52,新增題目,阿諾絲 波魯迪戈烏多,阿諾斯 波魯迪戈烏多,ios,zh,"iPhone11,6",新增題目,阿諾絲 波魯迪戈烏多,阿諾斯 波魯迪戈烏多,大便又來了\n,,[]


In [26]:
df['題目改進說明']

,題目改進說明
0,狗屎寫作咗個屎字糞
1,
2,跟鳳梨毫無關係
3,我第一次看到自己竟然可以出題目欸
4,
...,...
59,
60,
61,
62,


In [17]:
word_counts = Counter(all_words)
word_counts

Counter({'狗屎': 1,
         '寫作': 1,
         '鳳梨毫': 1,
         '無關': 1,
         '第一次': 1,
         '看到': 1,
         '自己': 1,
         '竟然': 1,
         '可以': 3,
         '出題': 1,
         '電腦': 1,
         '應該': 1,
         '改成': 1,
         '桌機': 1,
         '其實字': 1,
         '數無辜': 1,
         '一開始': 1,
         '出現': 1,
         '兩個': 2,
         '國中為': 1,
         '什麼': 3,
         '最後臥底': 1,
         '國中': 1,
         '這兩個': 2,
         '基本': 1,
         '合适': 1,
         '不太像': 1,
         '烏蘇拉是': 1,
         '快乐': 1,
         '根本': 1,
         '一樣': 1,
         '太像': 1,
         '奶牛': 1,
         '乳牛': 1,
         '接近': 1,
         '沒有': 1,
         '臥底': 2,
         '包差': 1,
         '太遠': 1,
         '詞太像': 1,
         '蔬菜': 1,
         '賽格威': 1,
         '看手機': 1,
         '刷視': 1,
         '頻刷': 1,
         '視頻': 1,
         '玩手': 1,
         '所以': 1,
         '這個': 1,
         '太多': 1,
         '玩家': 1,
         '東西': 1,
         '差太多': 2,
         '意思': 1,
         '差別

In [18]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}
filtered_word_counts

{'狗屎': 1,
 '寫作': 1,
 '鳳梨毫': 1,
 '無關': 1,
 '第一次': 1,
 '看到': 1,
 '自己': 1,
 '竟然': 1,
 '可以': 3,
 '出題': 1,
 '電腦': 1,
 '應該': 1,
 '改成': 1,
 '桌機': 1,
 '其實字': 1,
 '數無辜': 1,
 '一開始': 1,
 '出現': 1,
 '兩個': 2,
 '國中為': 1,
 '什麼': 3,
 '最後臥底': 1,
 '國中': 1,
 '這兩個': 2,
 '基本': 1,
 '合适': 1,
 '不太像': 1,
 '烏蘇拉是': 1,
 '快乐': 1,
 '根本': 1,
 '一樣': 1,
 '太像': 1,
 '奶牛': 1,
 '乳牛': 1,
 '接近': 1,
 '沒有': 1,
 '臥底': 2,
 '包差': 1,
 '太遠': 1,
 '詞太像': 1,
 '蔬菜': 1,
 '賽格威': 1,
 '看手機': 1,
 '刷視': 1,
 '頻刷': 1,
 '視頻': 1,
 '玩手': 1,
 '太多': 1,
 '玩家': 1,
 '東西': 1,
 '差太多': 2,
 '意思': 1,
 '差別': 1,
 '太大喉': 1,
 '改為': 1,
 '檳榔': 1,
 '两个': 1,
 '题目': 1,
 '答案': 1,
 '几乎': 1,
 '一样': 1,
 '知道': 1,
 '覺得': 1,
 '這啊': 1}

In [19]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [20]:
%%capture
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

In [21]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [22]:
from google.colab import userdata
apikey = userdata.get('geminiApiKey')

In [28]:
from IPython.display import Markdown
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-2.0-flash")

prompt = f"請摘要出這些題目評論的重點，並補上重點中的情緒表現，把最多人呈現的情緒及其相關的內容整併在一起，說明這些留言的風向：{df.to_string()}"

response = model.generate_content(prompt)
display(Markdown(response.text))

這些題目評論的重點可以歸納如下，並補上情緒表現：

**1. 對題目相似度/關聯性的質疑 (主要情緒：困惑、不解、不滿)**

*   許多評論指出題目中的兩個詞語太過相似、太接近 (例如：輔導老師/班導師、玩手機/看手機、可口可樂/百事可樂、生活費/零用錢、水壺/茶壺)。這顯示使用者認為題目不具備足夠的區辨度，難以作為臥底遊戲的題材。
*   部分評論認為題目中的詞語關聯性太弱或根本沒有關聯 (例如：草莓/鳳梨、臭豆腐/狗屎、書包/書)。這也引發使用者的困惑和質疑，認為這些題目不合理或不適合。
*   此外，也有評論指出題目中出現的詞語超出認知範圍，造成使用者的不解(例如：賽格威是三小、烏蘇拉是什麼)。

**2. 對題目適切性的質疑 (主要情緒：不認同、反感)**

*   有少部分評論帶有負面情緒，認為題目不雅或不恰當 (例如：臭豆腐/狗屎，使用「狗屎」字眼; 爸爸/SB)。
*   另有評論對於題目的意義不明感到不認同 (例如：1/1)。

**3.  對題目內容的建議 (主要情緒：建議、提供想法)**

*   部分評論針對現有題目提出修改建議，例如將「電腦」改為「桌機」，或將「奶牛」改為「乳牛」。
*   也有評論指出題目設定上的邏輯問題，例如「一開始出現了是兩個國中為什麼最後臥底是國中」。

**4.  個人感受 (主要情緒：喜悅、困惑)**

*   有使用者表達了第一次可以出題目的喜悅之情。
*   另有使用者對於題目隨機變換感到困惑。

**整體留言風向：**

整體而言，留言風向偏向**負面/質疑**。使用者主要針對題目的**相似度/關聯性**和**適切性**提出質疑，認為許多題目不夠好、不合理或不適合，帶有**困惑、不解、不滿**的情緒。另一方面，也有一些使用者會提出修改建議，或是分享個人感受，但整體而言，對題目品質的擔憂和批評是主要趨勢。
